In [ ]:
# Import stuff
import re
import numpy as np
from collections import deque
from copy import deepcopy

In [ ]:
# Get input
with open('input', 'r') as f:
    inp = [list(line.strip()) for line in f.readlines()]

In [ ]:
# Solve 1

# Find S
for x in range(len(inp)):
    for y in range(len(inp[0])):
        if inp[x][y] == 'S':
            S = (x, y)
            inp[x][y] = '|'
            break

# DFS
visited = set()
stack = deque()
stack.append(S)
path_len = 0

while len(stack) > 0:
    (x,y) = stack.pop()
    if (x,y) in visited:
        continue
    visited.add((x,y))
    path_len += 1

    pipe = inp[x][y]
    if pipe == '|':
        stack.append((x + 1, y))
        stack.append((x - 1, y))
    elif pipe == '-':
        stack.append((x, y + 1))
        stack.append((x, y - 1))
    elif pipe == 'F':
        stack.append((x + 1, y))
        stack.append((x, y + 1))
    elif pipe == '7':
        stack.append((x + 1, y))
        stack.append((x, y - 1))
    elif pipe == 'L':
        stack.append((x - 1, y))
        stack.append((x, y + 1))
    elif pipe == 'J':
        stack.append((x - 1, y))
        stack.append((x, y - 1))

print(path_len // 2)

In [ ]:
# Solve 2

class PathNode:
    def __init__(self, pos, dir):
        self.pos = pos
        self.dir = dir
    
    def __hash__(self):
        return hash(self.pos)
    
    def __eq__(self, other):
        return self.pos == other.pos

    def __repr__(self) -> str:
        return f'(p={self.pos},d={self.dir})'

def flood_fill(pos, inp, visited):
    num_tiles = 0
    stack = deque()
    stack.append(pos)

    while len(stack) > 0:
        pos = stack.pop()
        (x,y) = pos
        if pos in visited or inp[x][y] != '.':
            continue
        visited.add(pos)
        num_tiles += 1

        stack.append((x + 1, y))
        stack.append((x - 1, y))
        stack.append((x, y + 1))
        stack.append((x, y - 1))
    
    return num_tiles

# Find S
for x in range(len(inp)):
    for y in range(len(inp[0])):
        if inp[x][y] == 'S':
            S = (x, y)
            break

# DFS
start_dir = (1, 0)
visited = set()
stack = deque()
stack.append(PathNode(S, start_dir))
path_len = 0

while len(stack) > 0:
    path_node = stack.pop()
    if path_node in visited:
        continue
    visited.add(path_node)
    path_len += 1

    (x,y) = path_node.pos
    (dx,dy) = path_node.dir
    pipe = inp[x][y]

    if pipe == 'S':
        stack.append(PathNode((x + 1, y), start_dir))
    if pipe == '|':
        stack.append(PathNode((x + 1, y), (1, 0)))
        stack.append(PathNode((x - 1, y), (-1, 0)))
    elif pipe == '-':
        stack.append(PathNode((x, y + 1), (0, 1)))
        stack.append(PathNode((x, y - 1), (0, -1)))
    elif pipe == 'F':
        stack.append(PathNode((x + 1, y), (1, 0)))
        stack.append(PathNode((x, y + 1), (0, 1)))
    elif pipe == '7':
        stack.append(PathNode((x + 1, y), (1, 0)))
        stack.append(PathNode((x, y - 1), (0, -1)))
    elif pipe == 'L':
        stack.append(PathNode((x - 1, y), (-1, 0)))
        stack.append(PathNode((x, y + 1), (0, 1)))
    elif pipe == 'J':
        stack.append(PathNode((x - 1, y), (-1, 0)))
        stack.append(PathNode((x, y - 1), (0, -1)))

# Remove all parts not part of loop
for x in range(len(inp)):
    for y in range(len(inp[0])):
        if PathNode((x,y),(0,0)) not in visited:
            inp[x][y] = '.'
inp[S[0]][S[1]] = '|'

tiles_visited = set()
num_tiles = 0
for path_node in visited:
    (x,y) = path_node.pos
    (dx,dy) = path_node.dir
    if dx == 1: # Going down
        tiles = flood_fill((x, y - 1), inp, tiles_visited)
        if inp[x][y] == 'L':
            tiles += flood_fill((x + 1, y), inp, tiles_visited)
    elif dx == -1: # Going up
        tiles = flood_fill((x, y + 1), inp, tiles_visited)
        if inp[x][y] == '7':
            tiles += flood_fill((x - 1, y), inp, tiles_visited)
    elif dy == 1:  # Going right
        tiles = flood_fill((x + 1, y), inp, tiles_visited)
        if inp[x][y] == 'J':
            tiles += flood_fill((x, y + 1), inp , tiles_visited)
    elif dy == -1: # Going left
        tiles = flood_fill((x - 1, y), inp, tiles_visited)
        if inp[x][y] == 'F':
            tiles += flood_fill((x, y - 1), inp, tiles_visited)
    else:
        raise ValueError(f'Invalid node at {x}, {y}')
    num_tiles += tiles

print(num_tiles)